# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [3]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts url in it and check which other subreddit reposted that posts and how much:

In [4]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

for i in reddit.subreddit("conspiracy").top(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        subreddit_url = str(repost.subreddit)
        subreddit_url = "https://www.reddit.com/r/" + subreddit_url
        if subreddit_url in conspiracy_dict.keys():
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            conspiracy_dict[subreddit_url][1][0] +=1
        else:
            conspiracy_dict[subreddit_url]=[[],[1]]
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_top_url.csv',index=False)




# Creating the network:
Reading the csv we can now look for posts that have the same url as the ones in the file: this is done in order to later create a network where the subreddits will serve as nodes and the crossposts as edges between them (the more the crossposts the wider the edge).

In [4]:
df1 = pd.read_csv('results/conspiracy_top_url.csv')
id_to_analyse = []
for column in df1.columns:
    value = df1[column][1][1:-1]
    if int(value) >= 5:
        ind = column.index('/r/')
        id = column[ind+3:]
        id_to_analyse.append(id)

for subreddit in id_to_analyse:
    post_list=list()
    subreddit_list = list()
    conspiracy_dict=dict() 
    for i in reddit.subreddit(subreddit).top(limit=5000):
        post_list.append((i.title, i.score, i.url))


    for post in post_list:
        for repost in reddit.subreddit('all').search('url:'+post[2]):
            if repost.subreddit_id != "t5_"+subreddit: #cosa facciamo?
                subreddit_url = str(repost.subreddit)
                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                if subreddit_url in conspiracy_dict.keys():
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                    conspiracy_dict[subreddit_url][1][0] +=1
                else:
                    conspiracy_dict[subreddit_url]=[[],[1]]
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
    df = pd.DataFrame(conspiracy_dict)
    df.to_csv(r'results/'+subreddit+'.csv')

KeyboardInterrupt: 

-Il nome del subreddit di partenza é il nome del file <br>
-il nome del subreddit ripostante  e la riga 0 del csv <br>
-i permalink comuni tra i due sono la lista in riga 1

# Extracting comments
In order to perform the sentiment analysis and the topic modeling we first need to extract comments from the various subreddits' posts identified in the CSVs above.

In [ ]:
dir = "results"
to_scan = [] #iterate over this
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f[-4:] == ".csv":
        to_scan.append(dir+"/"+f) 

comment_dict=dict()

for file in to_scan:
    df = pd.read_csv(file)
    name_subreddit = file[7:-4]
    for column in df.columns:
        url_list = df[column][1]
        for permalink in url_list:
            
            submission=reddit.submission(url=permalink)
            comment_dict[name_subreddit +"--"+submission.subreddit]={permalink: []}
            comments = [submission.comments.replace_more(limit=None)]
            for comment in comments:
                comment_dict[subreddit][permalink][0].append(comment)


# DOBBIAMO ARRIVARE AD AVERE UN PERMALINK ASSOCIATO ALLA SENTIMENT DEI COMMENTI E DEL POST
